# LatentForge TTS 다중 GPU 학습 노트북 (Accelerate 사용)

이 노트북은 `accelerate` 라이브러리의 `notebook_launcher`를 사용하여 Jupyter 환경에서 다중 GPU 학습을 실행하는 방법을 보여줍니다.

`.py` 파일로 모듈화된 코드를 가져와서 사용하므로, 코드의 재사용성과 관리 용이성을 높일 수 있습니다.

- CUDA VISIBLE DEVICE 설정

In [1]:
import os

# CUDA 디바이스 설정 (torch 임포트 전에 해야 함!)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 사용할 GPU 번호들 #Ex) "0,1,2,3"

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Available GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: True
Available GPUs: 1
GPU 0: NVIDIA GeForce RTX 4070 Laptop GPU


## 1. 환경 설정 및 라이브러리 임포트

In [3]:
import os
import sys
from pathlib import Path
import torch
from accelerate import Accelerator, notebook_launcher
from tqdm.auto import tqdm

# 프로젝트의 src 폴더를 Python 경로에 추가
project_root = Path.cwd()
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# 필요한 모듈 임포트 (상대 임포트 문제를 피하기 위해 직접 경로 사용)
from config import TrainingConfig
from data_loading.dataset import create_dataloader
from models.tts_lora import TTSWithLoRA
from training.losses import CombinedLoss  # 직접 losses 모듈에서 가져오기
from peft import LoraConfig, TaskType

print("✅ 모든 모듈 임포트 성공!")

✅ 모든 모듈 임포트 성공!


In [4]:
import random
import numpy as np

def set_seed(seed: int = 42):
    """Set seed for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
# 전역 seed 설정
set_seed(42)

## 2. 학습 함수 정의

`accelerate`를 사용하려면 모든 학습 로직을 하나의 함수 안에 캡슐화해야 합니다. `notebook_launcher`가 이 함수를 각 GPU 프로세스에서 실행합니다.

In [6]:
def training_function(config: TrainingConfig):
    """다중 GPU 학습을 위한 메인 함수"""
    
    # 1. Accelerator 초기화
    # Accelerator가 자동으로 장치 할당, 모델 및 데이터 병렬 처리를 관리합니다.
    accelerator = Accelerator()
    print(f"Process {accelerator.process_index} starting on device: {accelerator.device}")

    # 2. 데이터 로더 생성
    # 메인 프로세스에서만 데이터를 다운로드하도록 설정 (accelerator.is_main_process)
    if accelerator.is_main_process:
        print("Creating dataloaders...")
    train_dataloader = create_dataloader(
        dataset_name=config.dataset_name,
        split="train.clean.100", # 데모를 위해 작은 데이터셋 사용
        batch_size=config.batch_size,
        sample_rate=config.sample_rate,
        cache_dir=config.cache_dir,
        seed = 42,
    )
    # val_dataloader = ... (필요시 추가)

    # 3. 모델, 옵티마이저, 손실 함수 초기화
    if accelerator.is_main_process:
        print("Initializing model, optimizer, and loss function...")
        
    lora_config = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        r=config.lora_r,
        lora_alpha=config.lora_alpha,
        lora_dropout=config.lora_dropout,
        target_modules=config.target_modules
    )

    hypernetwork_config = {
        'hidden_dim': config.hypernetwork_hidden_dim,
        'num_layers': config.hypernetwork_num_layers,
    }
    model = TTSWithLoRA(
        tts_model_name=config.tts_model_name,
        clap_model_name=config.clap_model_name,
        qwen_model_name=config.qwen_model_name,
        lora_config=lora_config,
        hypernetwork_config=hypernetwork_config,
        manual_target_modules=config.target_modules,
    )
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
    loss_fn = CombinedLoss(sample_rate=config.sample_rate)

    # 4. accelerator.prepare()로 모든 객체 래핑
    # 이 단계를 통해 모델, 옵티마이저, 데이터 로더가 분산 환경에 맞게 준비됩니다.
    model, optimizer, train_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader
    )

    # 5. 학습 루프
    if accelerator.is_main_process:
        print("Starting training...")
        
    progress_bar = tqdm(range(config.num_epochs * len(train_dataloader)), disable=not accelerator.is_local_main_process)
    model.train()
    
    for epoch in range(config.num_epochs):
        total_loss = 0
        for step, batch in enumerate(train_dataloader):
            # 데이터셋에서 필요한 정보 추출
            target_audio = batch['audio']  # 원본 음성
            ## 디버깅 코드 ---------------------------------------------
            # print("type(batch): ", type(batch))
            # print("batch: ", batch)
            content_text = batch['text']  # TTS에 입력할 대사
            
            # Speaker/Style prompt를 결합하여 CLAP 인코더에 전달
            speaker_descriptions = batch["speaker_descriptions"]
            
            # HyperNetwork가 CLAP 임베딩을 기반으로 LoRA 가중치를 생성하고,
            # 생성된 LoRA가 적용된 TTS 모델로 content_prompt를 음성으로 합성
            generated_audio = model(
                content_text=content_text,
                speaker_audio=target_audio,  # reference audio for CLAP audio encoding
                speaker_text=speaker_descriptions,  # combined speaker + style prompts
                sample_rate=config.sample_rate
            )
            
            # 합성된 음성과 원본 음성 비교하여 Loss 계산
            loss_dict = loss_fn(
                generated_audio=generated_audio,
                target_audio=target_audio,
                reference_audio=target_audio
            )
            loss = loss_dict['total']
            
            # 역전파 - HyperNetwork의 파라미터만 업데이트됨
            accelerator.backward(loss)
            
            optimizer.step()
            optimizer.zero_grad()
            
            progress_bar.update(1)
            progress_bar.set_postfix(loss=loss.item())
            total_loss += loss.item()
            
            # 로그 출력 (매 10 스텝마다)
            if step % 10 == 0 and accelerator.is_main_process:
                print(f"Epoch {epoch+1}, Step {step}: Loss = {loss.item():.4f}")
        
        # 에포크별 평균 손실 출력
        avg_loss = total_loss / len(train_dataloader)
        accelerator.print(f"Epoch {epoch+1}/{config.num_epochs}, Average Loss: {avg_loss:.4f}")

    # 학습 완료 후 모델 저장 (메인 프로세스에서만)
    if accelerator.is_main_process:
        print("Training finished. Saving model...")
        unwrapped_model = accelerator.unwrap_model(model)
        output_path = Path(config.output_dir) / "final_model.pt"
        output_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(unwrapped_model.state_dict(), output_path)
        print(f"Model saved to {output_path}")

In [7]:
# LoRA 타겟 설정
manual_targets = []

# 민관: 이유는 모르것는데, 18로 하면 오류 생김. 그래서 17로 해둠.
# 모델 구조 봤을 때는, 레이어가 18개 이기는 함.
for i in range(17):
   manual_targets.append(f"decoder.layers.{i}.cross_attention.k_proj")
   manual_targets.append(f"decoder.layers.{i}.cross_attention.o_proj")
   manual_targets.append(f"decoder.layers.{i}.cross_attention.q_proj")
   manual_targets.append(f"decoder.layers.{i}.cross_attention.v_proj")
   manual_targets.append(f"decoder.layers.{i}.self_attention.k_proj")
   manual_targets.append(f"decoder.layers.{i}.self_attention.o_proj")
   manual_targets.append(f"decoder.layers.{i}.self_attention.q_proj")
   manual_targets.append(f"decoder.layers.{i}.self_attention.v_proj")

print(manual_targets)

['decoder.layers.0.cross_attention.k_proj', 'decoder.layers.0.cross_attention.o_proj', 'decoder.layers.0.cross_attention.q_proj', 'decoder.layers.0.cross_attention.v_proj', 'decoder.layers.0.self_attention.k_proj', 'decoder.layers.0.self_attention.o_proj', 'decoder.layers.0.self_attention.q_proj', 'decoder.layers.0.self_attention.v_proj', 'decoder.layers.1.cross_attention.k_proj', 'decoder.layers.1.cross_attention.o_proj', 'decoder.layers.1.cross_attention.q_proj', 'decoder.layers.1.cross_attention.v_proj', 'decoder.layers.1.self_attention.k_proj', 'decoder.layers.1.self_attention.o_proj', 'decoder.layers.1.self_attention.q_proj', 'decoder.layers.1.self_attention.v_proj', 'decoder.layers.2.cross_attention.k_proj', 'decoder.layers.2.cross_attention.o_proj', 'decoder.layers.2.cross_attention.q_proj', 'decoder.layers.2.cross_attention.v_proj', 'decoder.layers.2.self_attention.k_proj', 'decoder.layers.2.self_attention.o_proj', 'decoder.layers.2.self_attention.q_proj', 'decoder.layers.2.sel

## 3. 학습 실행

`notebook_launcher`를 사용하여 위에서 정의한 `training_function`을 실행합니다. `num_processes` 인자를 통해 사용할 GPU 수를 지정할 수 있습니다.

- 원래 사용하던 학습 스크립트

In [8]:
# 학습 설정 로드
config = TrainingConfig(
    batch_size=1, # GPU 메모리에 맞게 조정
    num_epochs=1, # 데모용 에포크 수
    learning_rate=1e-5,
    target_modules=manual_targets
)

# 사용할 GPU 수
num_gpus = torch.cuda.device_count()
print(f"Found {num_gpus} GPUs.")

# notebook_launcher로 학습 시작
# 이 셀을 실행하면 지정된 수의 프로세스가 백그라운드에서 실행됩니다.
notebook_launcher(training_function, args=(config,), num_processes=num_gpus)

Found 1 GPUs.
Launching training on one GPU.
Process 0 starting on device: cuda
Creating dataloaders...


Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/116 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/116 [00:00<?, ?it/s]

...

[DEBUG][Hypernetwork] validated targets: 136 kept, 0 missing
   e.g. ['decoder.layers.0.cross_attention.k_proj', 'decoder.layers.0.cross_attention.o_proj', 'decoder.layers.0.cross_attention.q_proj', 'decoder.layers.0.cross_attention.v_proj']
Starting training...


  0%|          | 0/32182 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[DEBUG] --- Processing batch item 0 ---
[DEBUG] Speaker embedding shape: torch.Size([1, 1024])
[DEBUG][Hypernetwork] target modules count -> self_attn: 68, cross_attn: 68
[DEBUG] Generated LoRA weights shape (A, B for first module): torch.Size([1, 16, 1024]), torch.Size([1, 2048, 16])
[DEBUG] LoRA weights shape after un-batching: torch.Size([16, 1024]), torch.Size([2048, 16])
[DEBUG] --- Applying LoRA Weights ---
[DEBUG] LoRA weight keys: ['decoder.layers.0.cross_attention.k_proj', 'decoder.layers.0.cross_attention.o_proj', 'decoder.layers.0.cross_attention.q_proj', 'decoder.layers.0.cross_attention.v_proj', 'decoder.layers.0.self_attention.k_proj', 'decoder.layers.0.self_attention.o_proj', 'decoder.layers.0.self_attention.q_proj', 'decoder.layers.0.self_attention.v_proj', 'decoder.layers.1.cross_attention.k_proj', 'decoder.layers.1.cross_attention.o_proj', 'decoder.layers.1.cross_attention.q_proj', 'decoder.layers.1.cross_attention.v_proj', 'decoder.layers.1.self_attention.k_proj', 'd

KeyboardInterrupt: 